In [16]:
# Importing Libraries
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
import pandas as pd
import numpy as np
import string, os

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Load Dataset
all_headlines = []
curr_dir = 'drive/My Drive/newyork_headline/'
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_DF = pd.read_csv(curr_dir + filename)
        #print(article_DF)
        all_headlines.extend(list(article_DF.headline.values))
        break

In [20]:
len(all_headlines)

850

In [21]:
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

777

In [22]:
# Preprocessing of the Data

def clean_data(text_Data):
    text_Data = "".join(v for v in text_Data if v not in string.punctuation).lower()
    text_Data = text_Data.encode("utf8").decode("ascii",'ignore')
    return text_Data 

corpus = [clean_data(headline) for headline in all_headlines]
corpus[:10]

[' gop leadership poised to topple obamas pillars',
 'fractured world tested the hope of a young president',
 'little troublemakers',
 'angela merkel russias next target',
 'boots for a stranger on a bus',
 'molder of navajo youth where a game is sacred',
 'the affair season 3 episode 6 noah goes home',
 'sprint and mr trumps fictional jobs',
 'america  becomes a stan',
 'fighting diabetes and leading by example']

In [23]:
# Tokenization of words
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    Words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, Words

inp_sequences, Words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[73, 313],
 [73, 313, 616],
 [73, 313, 616, 3],
 [73, 313, 616, 3, 617],
 [73, 313, 616, 3, 617, 205],
 [73, 313, 616, 3, 617, 205, 314],
 [618, 38],
 [618, 38, 619],
 [618, 38, 619, 1],
 [618, 38, 619, 1, 206]]

In [24]:
def generate_padded_sequences(input_seq):
    max_sequence_length = max([len(x) for x in input_seq])
    input_seq = np.array(pad_sequences(input_seq, maxlen=max_sequence_length, padding='pre'))
    
    predictors, label = input_seq[:,:-1],input_seq[:,-1]
    label = ku.to_categorical(label, num_classes=Words)
    return predictors, label, max_sequence_length

predictors, label, max_sequence_length = generate_padded_sequences(inp_sequences)

In [25]:
def create_model(max_sequence_len, Words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # Add Input Embedding Layer
    model.add(Embedding(Words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(Words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = create_model(max_sequence_length, Words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 10)            22170     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2217)              223917    
Total params: 290,487
Trainable params: 290,487
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [27]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [28]:
print (generate_text("united states", 8, model, max_sequence_length))
print (generate_text("president trump", 7, model, max_sequence_length))
print (generate_text("donald trump", 7, model, max_sequence_length))
print (generate_text("india and china", 9, model, max_sequence_length))
print (generate_text("new york", 10, model, max_sequence_length))
print (generate_text("science and technology", 7, model, max_sequence_length))

United States Details Torture In Chinese Police If Act The
President Trump Denies Climate Change These Kids Die May
Donald Trump Aid Obama Teams Improve Food Sources In
India And China Bucking The Establishment And A Press Presidency Forged In
New York Today A Trumpless Tower With The New York Times Washington
Science And Technology Run Against Medical Advice Interference In Future
